In [25]:
import pandas as pd  # pip install pandas openpyxl
import os
import plotly.express as px  # pip install plotly-express
import plotly.graph_objects as go
from statsmodels.stats import proportion
import numpy as np
import numpy as np
import scipy.stats as st

In [26]:
df = pd.read_excel("15.06NRautoreduced to 100.xlsx", sheet_name="evaluations")



In [27]:
df["year"] = df["date"].dt.year
df = df[df["year"]>=2012]

In [28]:
df[["rating","company"]].groupby(["company"]).describe()["rating"][["mean","count","std"]].round(2)

,mean,count,std
company,,,
Bunq,3.01,271.0,1.93
C24 Bank,4.86,892.0,0.63
Commerzbank,1.38,1380.0,1.06
DKB,2.98,2996.0,1.86
Deutsche Bank,2.27,48.0,1.57
Deutsche Bank Service,1.82,642.0,1.56
Fidor Bank,3.06,2432.0,1.89
Hypovereinbank,1.56,502.0,1.32
ING,4.17,10330.0,1.49


In [29]:
sorted(df["year"].unique())

[2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [30]:
averagerating = (df.groupby(["banktype","year"],as_index=False).mean().round(2)[["banktype","rating","year"]])
banktype = averagerating["banktype"].unique()

averagerating_chart = go.Figure(data=[
    go.Bar(name='2015', x=banktype, y=averagerating[averagerating["year"]==2015]["rating"]),
    go.Bar(name='2016', x=banktype, y=averagerating[averagerating["year"]==2016]["rating"]),
    go.Bar(name='2017', x=banktype, y=averagerating[averagerating["year"]==2017]["rating"]),
    go.Bar(name='2018', x=banktype, y=averagerating[averagerating["year"]==2018]["rating"]),
    go.Bar(name='2019', x=banktype, y=averagerating[averagerating["year"]==2019]["rating"]),
    go.Bar(name='2020', x=banktype, y=averagerating[averagerating["year"]==2020]["rating"]),
    go.Bar(name='2021', x=banktype, y=averagerating[averagerating["year"]==2021]["rating"]),
    go.Bar(name='2022', x=banktype, y=averagerating[averagerating["year"]==2022]["rating"]),])
averagerating_chart.update_layout(barmode='group')
averagerating_chart.show()

In [31]:
#yearly_review_amounts = df.groupby(pd.Grouper(key="date",freq="Y")).count().reset_index(level=0)
yearly_review_amounts = (df.groupby(["year"],as_index=False).count().round(2)[["banktype","year"]])

In [32]:
yearly_review_amounts.head()

,banktype,year
0,137,2012
1,100,2013
2,658,2014
3,611,2015
4,1750,2016


In [36]:
yearly_amount_chart= px.line(yearly_review_amounts,x="year",y="banktype")
yearly_amount_chart.update_layout(title='Amount of new Reviews per Year',
           xaxis_title='Time', yaxis_title='Amount of new Reviews',title_x=0.5,width=900)

In [37]:
length_rating = go.Figure()
length_rating.add_trace(go.Box(x=df["rating"],
    y=df["length of review"],
    name="Onl",
    boxpoints="suspectedoutliers", # no data points
    marker_color='rgb(9,56,125)',
    line_color='rgb(9,56,125)'
))
length_rating.update_layout(title_text="Length of Reviews by Rating",title_x=0.5,xaxis_title='Rating', yaxis_title='Length of Review',width=900)

In [11]:
dimensiondf = df[df["customer dimension"]!="not classified"]
#customerdimension_barchart = px.bar(dimensiondf, x= "customer dimension",color="banktype",barmode="group")
#customerdimension_barchart.show()

In [12]:
dimensiondf.groupby("banktype").count()

,Spalte1,company,date,headline,body,rating,length of review,originalbody,word count,topic,customer dimension,year
banktype,,,,,,,,,,,,
Classic Private Bank,2407,2407,2407,2407,2407,2407,2407,2407,2407,2407,2407,2407
Cooperative banks,1482,1482,1482,1482,1482,1482,1482,1482,1482,1482,1482,1482
Directbank,5187,5187,5187,5187,5187,5187,5187,5187,5187,5187,5187,5187
Fintech,4244,4244,4244,4244,4244,4244,4244,4244,4244,4244,4244,4244
Saving Banks,282,282,282,282,282,282,282,282,282,282,282,282


In [13]:
banktypes = dimensiondf["banktype"].unique()
dimensions = dimensiondf["customer dimension"].unique()

proportions = []
error = []
customerdimension = []
banktypel = []
means = []
counts = []
error_mean = []

for banktype in banktypes:
    for dimension in dimensions:
        mean = dimensiondf[(df["banktype"]==banktype)& (df["customer dimension"]==dimension)]["rating"].mean()
        my,mx =st.norm.interval(alpha=0.95,loc=np.mean(dimensiondf[(df["banktype"]==banktype)& (df["customer dimension"]==dimension)]["rating"]))
        count = dimensiondf[(df["banktype"]==banktype)& (df["customer dimension"]==dimension)]["rating"].count()
        y,x =proportion.proportion_confint(count=dimensiondf[(df["banktype"]==banktype)& (df["customer dimension"]==dimension)]["body"].count(),    # Number of "successes"
                   nobs=dimensiondf[(df["banktype"]==banktype)]["body"].count(),    # Number of trials
                   alpha=(1 - 0.95))
        e = (x+y)/2-y
        p = (x+y)/2
        em = (mx+my)/2-my
        mm = (mx+my)/2
        proportions.append(p)
        error.append(e)
        customerdimension.append(dimension)
        banktypel.append(banktype)
        means.append(mean)
        error_mean.append(em)
        counts.append(count)
        


proportions_bydimension = pd.DataFrame({"dimension":customerdimension,'banktype': banktypel, 'proportion': proportions, 'error proportions': error,"mean" :means,"count":counts,"error mean":error_mean})



C:\Users\dusti\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:14: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\dusti\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:15: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\dusti\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:16: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\dusti\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:17: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\dusti\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:18: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [14]:
st.norm.interval(alpha=0.95,loc=np.mean(dimensiondf[(df["banktype"]=="Fintech")& (df["customer dimension"]=="App")]["rating"]))

C:\Users\dusti\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



(2.2717270940218635, 6.191655063101972)

In [15]:
gr_dimensiondf =dimensiondf.groupby(["customer dimension"],as_index=False).count()[["customer dimension","company"]]
total_reviews = gr_dimensiondf["company"].sum()
gr_dimensiondf['perc'] = gr_dimensiondf['company'].apply(lambda x: x*100/total_reviews.sum())
gr_dimensiondf = gr_dimensiondf.sort_values(by=["perc"],ascending=False)


In [16]:
dimensions_chart= px.bar(gr_dimensiondf,y="customer dimension",x="perc",orientation='h')
dimensions_chart.update_layout(title='Customer Reviews by Percentage',
           xaxis_title='Customer Dimension', yaxis_title='Percentage of Reviews',title_x=0.,)

In [17]:
relevant_proportions_bydimension = proportions_bydimension[proportions_bydimension["proportion"]>=0.01] 
proportions_chart = px.scatter(relevant_proportions_bydimension, x="dimension", y="proportion", color="banktype",error_y="error proportions")
#error_y="error"
proportions_chart.update_layout(legend=dict(
    yanchor="bottom",
    y=-1.0,
    xanchor="left",
    x=0.01
))
proportions_chart.show()

In [38]:

chisquaretable= pd.crosstab(dimensiondf["banktype"],dimensiondf["customer dimension"],normalize="index",)



In [37]:
from scipy.stats import chi2_contingency

In [39]:
chi2_contingency(chisquaretable)

(2.8994436651468605,
 1.0,
 84,
 array([[0.00700721, 0.00728238, 0.00777766, 0.0667008 , 0.13276023,
         0.00756936, 0.01687862, 0.06665541, 0.10891169, 0.02618288,
         0.05305042, 0.05746286, 0.13191256, 0.00820151, 0.01141513,
         0.04935074, 0.02758915, 0.10766842, 0.00856289, 0.05972786,
         0.01450455, 0.02282767],
        [0.00700721, 0.00728238, 0.00777766, 0.0667008 , 0.13276023,
         0.00756936, 0.01687862, 0.06665541, 0.10891169, 0.02618288,
         0.05305042, 0.05746286, 0.13191256, 0.00820151, 0.01141513,
         0.04935074, 0.02758915, 0.10766842, 0.00856289, 0.05972786,
         0.01450455, 0.02282767],
        [0.00700721, 0.00728238, 0.00777766, 0.0667008 , 0.13276023,
         0.00756936, 0.01687862, 0.06665541, 0.10891169, 0.02618288,
         0.05305042, 0.05746286, 0.13191256, 0.00820151, 0.01141513,
         0.04935074, 0.02758915, 0.10766842, 0.00856289, 0.05972786,
         0.01450455, 0.02282767],
        [0.00700721, 0.00728238, 0.007

In [18]:
proportions_chart = px.bar(relevant_proportions_bydimension, x="dimension", y="mean", color="banktype",barmode="group"
                 )

proportions_chart.show()

In [19]:
proportions_chart = px.scatter(relevant_proportions_bydimension, x="dimension", y="mean", color="banktype",error_y="error mean")
proportions_chart.show()

In [20]:
dimensiondf["Satisfaction"] = np.where(dimensiondf["rating"]>=4, "Satisfied","Unsatisifed")
grcd_dimensiondf =dimensiondf.groupby(["customer dimension","Satisfaction"],as_index=False).count()[["customer dimension","Satisfaction","company"]]


C:\Users\dusti\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



dimensiondf["percentage"] = df.groupby("")

In [21]:
grcd_dimensiondf['perc'] = grcd_dimensiondf.groupby('customer dimension')['company'].apply(lambda x: x*100/x.sum())

In [22]:
fig = px.bar(
    grcd_dimensiondf,x="customer dimension",y="perc",color="Satisfaction",
       # if values in column z = 'some_group' and 'some_other_group'
    color_discrete_map={
        'Unsatisifed': 'firebrick',
        'Satisfied': 'forestgreen'
    }
)
fig.show()

In [23]:

yearly_dimensiondf = pd.DataFrame(columns=["customer dimension","company","year","perc"])

for year in dimensiondf["year"].unique():

    gryr_dimensiondf = dimensiondf[dimensiondf["year"]==year]
    gryr_dimensiondf =gryr_dimensiondf.groupby(["customer dimension"],as_index=False).count()[["customer dimension","company"]]
    gryr_dimensiondf["year"] = year
    total_reviews = gryr_dimensiondf["company"].sum()
    gryr_dimensiondf['perc'] = gryr_dimensiondf['company'].apply(lambda x: x*100/total_reviews.sum())
    yearly_dimensiondf = yearly_dimensiondf.append(gryr_dimensiondf,ignore_index=True)

#yearly_dimensiondf =gryr_dimensiondf.groupby(["customer dimension"],as_index=False).count()[["customer dimension","year","company","perc"]]
yearly_dimensiondf[yearly_dimensiondf["year"]==2016]["perc"].sum()

100.00000000000001

In [24]:
dimensiondf["yearspan"] = np.where((dimensiondf["year"]>=2014) &(dimensiondf["year"]<=2016), "2014-2016"
,np.where((dimensiondf["year"]>=2017) &(dimensiondf["year"]<=2019), "2017-2019",
np.where((dimensiondf["year"]>=2020) &(dimensiondf["year"]<=2022), "2020-2022","")))

C:\Users\dusti\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [25]:

yearly_dimensiondf = pd.DataFrame(columns=["customer dimension","company","yearspan","perc"])

for yearspan in dimensiondf["yearspan"].unique():

    gryr_dimensiondf = dimensiondf[dimensiondf["yearspan"]==yearspan]
    gryr_dimensiondf =gryr_dimensiondf.groupby(["customer dimension"],as_index=False).count()[["customer dimension","company"]]
    gryr_dimensiondf["yearspan"] = yearspan
    total_reviews = gryr_dimensiondf["company"].sum()
    gryr_dimensiondf['perc'] = gryr_dimensiondf['company'].apply(lambda x: x*100/total_reviews.sum())
    yearly_dimensiondf = yearly_dimensiondf.append(gryr_dimensiondf,ignore_index=True)

#yearly_dimensiondf =gryr_dimensiondf.groupby(["customer dimension"],as_index=False).count()[["customer dimension","year","company","perc"]]
yearly_dimensiondf[yearly_dimensiondf["yearspan"]=="2020-2022"]["perc"].sum()


99.99999999999999

In [69]:
fig = px.bar(yearly_dimensiondf.sort_values("yearspan"),x="customer dimension",y="perc",color="yearspan",barmode="group")

#error_y="error"
"""
fig.update_layout(dict(updatemenus=[
                        dict(
                            type = "buttons",
                            direction = "left",
                            buttons=list([
                                dict(
                                    args=["visible", "legendonly"],
                                    label="Deselect All",
                                    method="restyle"
                                ),
                                dict(
                                    args=["visible", True],
                                    label="Select All",
                                    method="restyle"
                                )
                            ]),
                            pad={"r": 10, "t": 10},
                            showactive=False,
                            x=1,
                            xanchor="right",
                            y=1.1,
                            yanchor="top"
                        ),
                    ]
              ))
              """
fig.show()

SyntaxError: EOL while scanning string literal (<ipython-input-69-fdb990fdf5ad>, line 1)

In [27]:
yearly_dimensiondf = pd.DataFrame(columns=["customer dimension","company","year","perc"])

for year in dimensiondf["year"].unique():

    gryr_dimensiondf = dimensiondf[dimensiondf["year"]==year]
    gryr_dimensiondf =gryr_dimensiondf.groupby(["customer dimension"],as_index=False).count()[["customer dimension","company"]]
    gryr_dimensiondf["year"] = year
    total_reviews = gryr_dimensiondf["company"].sum()
    gryr_dimensiondf['perc'] = gryr_dimensiondf['company'].apply(lambda x: x*100/total_reviews.sum())
    yearly_dimensiondf = yearly_dimensiondf.append(gryr_dimensiondf,ignore_index=True)

#yearly_dimensiondf =gryr_dimensiondf.groupby(["customer dimension"],as_index=False).count()[["customer dimension","year","company","perc"]]
#yearly_dimensiondf[yearly_dimensiondf["yearspan"]=="2020-2022"]["perc"].sum()

In [28]:
fig = px.bar(yearly_dimensiondf.sort_values("year"),x="customer dimension",y="perc",color="year",barmode="group")
fig.show()
#error_y="error"

In [29]:
dfsentiment = pd.read_excel("sentimentanalysisfordimensions15.06.xlsx",sheet_name="Treemap")

In [17]:
dftreemap = px.data.tips()

fig = px.treemap(dfsentiment, path=[px.Constant("all"), 'Dimension', 'semantic', 'Word'], values='count')
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict( l=25, r=25, b=25))
fig.show()

In [80]:
with pd.ExcelWriter('summarytables2.xlsx') as writer:  
    gr_dimensiondf.to_excel(writer, sheet_name='dimensions')
    proportions_bydimension.to_excel(writer, sheet_name='proportionsbydimension')
    yearly_dimensiondf.to_excel(writer, sheet_name='yearlydimensiondf')

In [38]:
from bertopic import BERTopic
topic_model = BERTopic.load("15.06NRautoreduced to 100")



In [36]:
topic_model.visualize_heatmap()

In [39]:
barcharttopicmodel = topic_model.visualize_barchart(n_words=5,top_n_topics=10)
barcharttopicmodel.show()

In [182]:
visualizedtopics = topic_model.visualize_topics()
visualizedtopics.write_html("visualizedtopics.html")

In [32]:
topic_model.visualize_term_rank()

In [29]:
topic_model.visualize_distribution(probabilities[0])

NameError: name 'probabilities' is not defined

In [183]:
topic_freq = topic_model.get_topic_freq()
outliers = topic_freq['Count'][topic_freq['Topic']==-1].iloc[0]
print(f"{outliers} documents have not been classified")
print(f"The other {topic_freq['Count'].sum() - outliers} documents are {topic_freq['Topic'].shape[0]-1} topics")

24766 documents have not been classified
The other 14669 documents are 80 topics


In [83]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(horizontal_spacing=0.09,rows=2, cols=4, shared_yaxes=False,subplot_titles=("Topic 0", "Topic 1","Topic 2","Topic 3","Topic 4","Topic 5","Topic 6","Topic 7"))

fig.add_trace(go.Bar(orientation='h',y=["more", "working","online" ,"banking","app"], x=[0.010, 0.011, 0.017,0.019,0.03],
                    ),
              row=1, col=1)
fig.add_trace(go.Bar(orientation='h',y=["employee", "catastrophe","customer" ,"worst","bank"], x=[0.09, 0.011, 0.013,0.018,0.022],
                    ),
              row=1, col=2)
fig.add_trace(go.Bar(orientation='h',y=["advisery", "advise","competent" ,"telefon","friendly"], x=[0.028, 0.0287, 0.0345,0.036,0.037],
                    ),
              row=1, col=3)
fig.add_trace(go.Bar(orientation='h',y=["inquery", "answered","question" ,"fast","answer"], x=[0.048, 0.063, 0.0688,0.089,0.09],
                    ),
              row=1, col=4)
fig.add_trace(go.Bar(orientation='h',y=["reach", "telephon","waiting loop" ,"minutes","hotline"], x=[0.014, 0.016, 0.018,0.023,0.031],
                    ),
              row=2, col=1)
fig.add_trace(go.Bar(orientation='h',y=["fast", "friendly","good" ,"competent","advisery"], x=[0.045, 0.073, 0.099,0.13,0.3],
                    ),
              row=2, col=2)
fig.add_trace(go.Bar(orientation='h',y=["good", "clear","banking" ,"super","app"], x=[0.019, 0.0224, 0.0228,0.028,0.09],
                    ),
              row=2, col=3)
fig.add_trace(go.Bar(orientation='h',y=["so far", "completly","super" ,"absolutly","satisfied"], x=[0.028, 0.029, 0.034,0.05,0.31],
                    ),
              row=2, col=4)




fig.update_layout(coloraxis=dict(colorscale='Bluered_r'), showlegend=False,width=1000,title="Topic Word scores",title_x=0.5)
fig.show()